In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

from prophet import Prophet
from prophet.plot import plot_plotly
from prophet.plot import add_changepoints_to_plot

In [6]:
# Reading the dataset
dataset_path = "/kaggle/input/generaldataset/cpi_canada.csv"
df = pd.read_csv(dataset_path)
df.head(5)

,REF_DATE,GEO,DGUID,Products and product groups,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2010-09,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.2,116.9,NaN,NaN,NaN,1
1,2010-10,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.2,117.4,NaN,NaN,NaN,1
2,2010-11,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.2,117.5,NaN,NaN,NaN,1
3,2010-12,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.2,117.5,NaN,NaN,NaN,1
4,2011-01,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.2,117.8,NaN,NaN,NaN,1


In [ ]:
my_model = Prophet(interval_width=0.95,daily_seasonality=True)
my_model.fit(df)

#freq = 'S' Seconds
#freq = "M" Month
#freq = "min" minute
#freq = "D" day
future_dates = my_model.make_future_dataframe(periods=200 , freq = 'D' , include_history = 'TRUE')
forecast = my_model.predict(future_dates)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head()
fig1 = my_model.plot(forecast)
fig2 = my_model.plot_components(forecast)

In [ ]:
forecast[(forecast['ds'] >= '02/01/2024') & (forecast['ds'] <= '03/31/2024')].head(10)

In [ ]:
fig = my_model.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), my_model, forecast)

In [ ]:
my_model.changepoints